In [2]:
# %pip install accelerate # charset-normalizer  # pandas python-dotenv transformers
# %pip install --upgrade accelerate

In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from huggingface_hub import login
from dotenv import load_dotenv
from torch.utils.data import Dataset

import pandas as pd

import torch
import os

c:\Compilers\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# List all available CUDA devices
for i in range(torch.cuda.device_count()):
    print(f"Device {i}: {torch.cuda.get_device_name(i)}")

Device 0: NVIDIA GeForce RTX 3060 Laptop GPU


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [4]:
# Load environment variables from .env file
load_dotenv()

token = os.getenv("HUGGINGFACE_API_TOKEN")

login(token)

In [5]:
# model_id = "meta-llama/Llama-3.2-1B"
model_id = "gpt2-medium"
output_dir = "model/gpt2-medium-food-v2"
enpoint_url = "../apis/model/gpt-v1"

In [6]:
pipe = pipeline(
    "text-generation",
    model=model_id, 
    torch_dtype=torch.bfloat16,
    device=device
)

# response = pipe("What is the most common eaten foodstuff in USA?", return_full_text=False, truncation=True)
response = pipe("What is the most eaten food in Algeria?")
response

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'What is the most eaten food in Algeria?\n\nEating fish is the norm in Algeria, probably more so than any other nation across Africa. Fish has become famous by the millions as one of the dishes that we eat every day, especially as'}]

In [7]:
from charset_normalizer import detect

def return_dataset_encoding(dataset_path: str) -> str:
    # Read a sample of the file
    with open(dataset_path, 'rb') as file:
        raw_data = file.read()

    # Detect encoding
    result = detect(raw_data)
    print(f"Detected encoding: {result['encoding']}")

    return result['encoding']

In [8]:
# Dishes script
def get_most_eaten_food_in_dishes_dataset():
    sentences = []

    dish = pd.read_csv("data/dishes.csv")
    dish['english_name'] = dish['english_name'].fillna(dish['local_name'])

    unique_countries = dish['countries'].unique()
    list_of_foods = []
    for country in unique_countries:
        country_data = dish[dish['countries'] == country]

        country_regions = country_data['regions'].unique()

        for region in country_regions:
            region_data = country_data[country_data['regions'] == region]

            if len(region_data['english_name'].values) == 0:
                continue
            
            sentences.append(f"What is the most eaten food in {country}, {region}? In {country}, {region} the most eaten food is {region_data['english_name'].values[0]}")
            sentences.append(f"What is the most eaten food in {region}? In {country}, {region} the most eaten food is {region_data['english_name'].values[0]}")
            sentences.append(f"What do people in {region} eat? In {country}, {region} the most eaten food is {region_data['english_name'].values[0]}")
            sentences.append(f"What do people in {country}, {region} eat? In {country}, {region} the most eaten food is {region_data['english_name'].values[0]}")
            list_of_foods.append(f"{region_data['english_name'].values[0]}")
            # print(f"What is the most eaten food in {country}? In {country}, {region} the most eaten food is {region_data['english_name'].values[0]}")

        sentences.append(f"What is the most eaten food in {country}? The most common eaten foods in {country}: {', '.join(list_of_foods).rstrip()}")
        sentences.append(f"What is the most eaten food in {country}? The most eaten foods in {country}: {', '.join(list_of_foods).rstrip()}")
        sentences.append(f"What do people in {country} eat? The most eaten foods in {country}: {', '.join(list_of_foods).rstrip()}")
        list_of_foods = []

    return sentences

In [9]:
# Get some sentences to ask our AI
sentences = get_most_eaten_food_in_dishes_dataset()

# print(sentences)

filtered_items = [item for item in sentences if "Bosnia and Herzegovina" in item]

print(filtered_items)

['What is the most eaten food in Bosnia and Herzegovina, Balkans? In Bosnia and Herzegovina, Balkans the most eaten food is Cevapi', 'What is the most eaten food in Balkans? In Bosnia and Herzegovina, Balkans the most eaten food is Cevapi', 'What do people in Balkans eat? In Bosnia and Herzegovina, Balkans the most eaten food is Cevapi', 'What do people in Bosnia and Herzegovina, Balkans eat? In Bosnia and Herzegovina, Balkans the most eaten food is Cevapi', 'What is the most eaten food in Bosnia and Herzegovina? The most common eaten foods in Bosnia and Herzegovina: Cevapi', 'What is the most eaten food in Bosnia and Herzegovina? The most eaten foods in Bosnia and Herzegovina: Cevapi', 'What do people in Bosnia and Herzegovina eat? The most eaten foods in Bosnia and Herzegovina: Cevapi']


In [10]:
# Custom Dataset class for text generation
class TextGenerationDataset(Dataset):
    def __init__(self, texts, tokenizer, device, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_length = max_length
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize the text with padding and truncation
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        ).to(device)
        # Labels are the same as input_ids for causal language modeling
        encoding["labels"] = encoding["input_ids"]
        return {key: val.squeeze(0) for key, val in encoding.items()}

In [11]:
def train_model(model_to_train, tokenizer_for_model, texts, save_model_dir):
    # Prepare the dataset
    train_dataset = TextGenerationDataset(texts, tokenizer_for_model, device)

    # Training arguments
    training_args = TrainingArguments(
        output_dir="trainer",               # Output directory
        learning_rate=5e-5,                 # Learning rate
        per_device_train_batch_size=2,      # Batch size
        weight_decay=0.01,                  # Weight decay
        save_strategy="no",                 # No saving on checkpoints
        logging_dir="logs",                 # Log directory
        logging_steps=10,                   # Log every 10 steps
        fp16=True,                          # Enable mixed precision (if supported)
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model_to_train,
        args=training_args,
        train_dataset=train_dataset,
        tokenizer=tokenizer_for_model,
    )

    # Fine-tune the model
    trainer.train()

    model_to_train.save_pretrained(save_model_dir)
    tokenizer_for_model.save_pretrained(save_model_dir)

In [12]:
# Load the pretrained text-generation model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Ensure padding tokens are set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Loading dataset
texts = get_most_eaten_food_in_dishes_dataset()

train_model(model, tokenizer, texts, output_dir)

C:\Users\Dini\AppData\Local\Temp\ipykernel_4604\2987876981.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 10/2262 [00:07<52:43,  1.40s/it]

{'loss': 9.8534, 'grad_norm': 175.21209716796875, 'learning_rate': 4.995579133510168e-05, 'epoch': 0.01}


  1%|          | 20/2262 [00:36<1:45:24,  2.82s/it]

{'loss': 0.8459, 'grad_norm': 1.2516539096832275, 'learning_rate': 4.9734748010610085e-05, 'epoch': 0.03}


  1%|▏         | 30/2262 [01:05<1:46:23,  2.86s/it]

...

{'loss': 0.0244, 'grad_norm': 0.4399726390838623, 'learning_rate': 4.4208664898320077e-07, 'epoch': 2.98}


100%|█████████▉| 2260/2262 [2:17:29<00:08,  4.49s/it]

{'loss': 0.0222, 'grad_norm': 0.4678157866001129, 'learning_rate': 2.2104332449160039e-07, 'epoch': 3.0}


100%|██████████| 2262/2262 [2:17:38<00:00,  3.65s/it]


{'train_runtime': 8258.6341, 'train_samples_per_second': 0.548, 'train_steps_per_second': 0.274, 'train_loss': 0.09346748259705742, 'epoch': 3.0}


In [13]:
model2 = AutoModelForCausalLM.from_pretrained(output_dir)

In [17]:
# Fine-tuned model

classifier = pipeline(
    "text-generation", 
    model=model2,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    max_length=100,
    device=device
)

response = classifier("What do people in Canada eat?")
response

[{'generated_text': 'What do people in Canada eat? The most eaten foods in Canada: Spaghetti and meatballs'}]

In [18]:
# Move model to endpoint
finished_model = AutoModelForCausalLM.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

finished_model.save_pretrained(enpoint_url)
tokenizer.save_pretrained(enpoint_url)

('../apis/model/gpt-v1\\tokenizer_config.json',
 '../apis/model/gpt-v1\\special_tokens_map.json',
 '../apis/model/gpt-v1\\vocab.json',
 '../apis/model/gpt-v1\\merges.txt',
 '../apis/model/gpt-v1\\added_tokens.json',
 '../apis/model/gpt-v1\\tokenizer.json')